In [2]:
'''
Assignment 9
'''
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as sci
import math as mat
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import Markdown as md
import pystan
#from psis import psisloo

In [3]:
#
#Stan Code
#
# Hierarchical STAN model
factory_hierarchical_stan_code='''
data {
    int<lower=0> N; // number of data points
    int<lower=0> K; // number of groups
    int<lower=1,upper=K> x[N]; // group indicator
    vector[N] y; //
}
parameters {
    real mu0; // prior mean
    real<lower=0> sigma0; // prior std
    vector[K] mu; // group means
    real<lower=0> sigma; // common std
}
model {
    mu ~ normal(mu0, sigma0); // population prior with unknown parameters
    y ~ normal(mu[x], sigma);
}
generated quantities {
    real ypred6;
    real mu7;
    vector[N] log_lik;
    ypred6 = normal_rng(mu[6], sigma);
    mu7 = normal_rng(mu0, sigma0);
    for (i in 1:N)
    log_lik[i] = normal_lpdf(y[i] | mu[x[i]], sigma);
}
'''
sm_factory_hierarchical = pystan.StanModel(model_code=factory_hierarchical_stan_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_61a2bb39b7a6ed20c5bf761dfc63c539 NOW.


CompileError: command 'gcc' failed with exit status 4

In [ ]:
#Computation
#
y=np.array(([83 ,117, 101, 105, 79, 57],
[92, 109, 93, 119, 97, 92],
[92, 114, 92, 116, 103, 104],
[46, 104, 86, 102, 79, 77],
[67, 87, 67, 116, 92, 100] ))

y=np.reshape(y, 30, order='F')
x=np.concatenate( (np.ones(5) , 2*np.ones(5) , 3*np.ones(5), 4*np.ones(5),5*np.ones(5),6*np.ones(5)) ).astype(int)

factory_hierarchical_dat = {'y':y,
                            'x': x,
                            'N': 30,
                            'K': 6
                           }

# Hierarchical
fit_hierarchical = sm_factory_hierarchical.sampling(data=factory_hierarchical_dat, n_jobs=-1)
mu7_hierarchical=fit_hierarchical.extract()['mu7']
ypred6_hierarchical=fit_hierarchical.extract()['ypred6']
mu_hierarchical=fit_hierarchical.extract()['mu']
log_lik=fit_hierarchical.extract()['log_lik']
psis=psisloo(log_lik)
print("PSIS-LOO value:", psis[0])
#Estimate of the effective number of parameters
S=np.size(log_lik,0)
lppd=sum(np.log([1/S*sum(np.exp(col)) for col in log_lik.T]))
p_loocv=lppd-psis[0]
print("p_loo-cv estimated effective number of parameters:",p_loocv)

#Plotting
plt.hist(psis[2],bins=np.linspace(0,1,11))
plt.show()